# Doing Data Science - Tutorial on K-Nearest Neighbors
- The k-NN algorithm is a nonparametric approach to classification.
- K-NN is an algorithm that can be used when you have a bunch of objects that have been classified or labeled in some way, and other similar objects that haven't gotten classified or labeled yet, and you want a way to automatically label them.
- As a classification algorithm, it can be applied where 'linear-regression-with-a-threshold' cannot, such as when the labels do not take a continuous value scale like a credit score.
- The intution behind k-NN is to consider the most similar other items defined in terms of their attributes, look at their labels, and give the unassigned item the majority vote. If there's a tie, you randomly select among the labels that have tied for first.

**Two Decisions to be made**
1. how do you define similarity or closeness?
2. how many neighbors should vote? This value is *k*

**k-NN process overview**
1. Decide on your similarity or distance metric.
2. Split the original labeled dataset into training and test data.
3. Pick an evaluation metric. (Misclassification rate is a good one.)
4. Run k-NN a few times, changing *k* and checking the evaluation measure.
5. Optimize *k* by picking the one with the best evaluation measure.
6. Once you've chosen *k*, use the same training set and now create a new test set with people's ages and incomes that you have no labels for, and want to predict.

**Notable similarity metrics**
- Euclidean distance
- Cosine similarity
- Jaccard distance or similarity (Tanimoto)
- Mahalanobis distance
- Hamming distance
- Manhattan distance

In classification, a balance has to be struck between being *sensitive* to the reality of the data (true positive or recall) and being *specific* (true negative) to the classification with respect to the categories of the data.

- Code tutorial from [here](https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/) 
- Data: [Iris dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data)

In [1]:
import csv
import random
import math
import operator

**1. Handle Data**

In [155]:
def loadDataset(filename, split, trainingSet, testSet):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        # shuffle the dataset
        for i in range(5):
            random.shuffle(dataset)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y]= float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])

In [154]:
trainingSet, testSet = [], []
loadDataset("iris.data",0.66,trainingSet,testSet)

**2. Similarity**

In [11]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x]-instance2[x]),2)
    return math.sqrt(distance)

In [12]:
# distance test
data1 = [2,2,2,'a']
data2 = [4,4,4,'b']
distance = euclideanDistance(data1, data2, 3)
print("Distance:",str(distance))

Distance: 3.4641016151377544


**3. Neighbors**

The `getNeighbors()` function returns *k* most similar neighbors from the training set for a given test instance, using the already defined `euclideanDistance()`function

In [25]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [26]:
# neighbors test
trainSet = [[2,2,2,'a'],[4,4,4,'b']]
testInstance = [5,5,5]
k=1
neighbors = getNeighbors(trainingSet, testInstance, 1)
print(neighbors)

[[5.2, 4.1, 1.5, 0.1, 'Iris-setosa']]


**4. Response**

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.
We can do this by allowing each neighbor to vote for their class attribute, and take the majority vote as the prediction

In [30]:
def getResponseMajorityVote(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(),key=operator.itemgetter(1),reverse=True)
    return sortedVotes[0][0]

In [31]:
# majority vote test
neighbors = [[1,1,1,'a'],[2,2,2,'a'],[3,3,3,'b']]
response = getResponseMajorityVote(neighbors)
print(response)

a


**5. Accuracy**

We have all teh pieces of the kNN algorithm in place. An important remaining concern is how to evaluate the accuracy of predictions.

Below, the `getPredictionAccuracy()` function sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [99]:
def getPredictionAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
#     print("num of predictions:",len(predictions),"number correct:",correct)
    return (correct/float(len(testSet))) * 100.0

In [100]:
# accuracy test
testSet = [[1,1,1,'a'],[2,2,2,'a'],[3,3,3,'b']]
predictions = ['a','a','a']
accuracy = getPredictionAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


**6. Main - putting it all together**

In [158]:
def main():
    # prepare data
    trainingSet = []
    testSet = []
    split = 0.67
    loadDataset("iris.data",split,trainingSet,testSet)
    print("Training set:",len(trainingSet))
    print("Test set:",len(testSet))
    
    #generate predictions
    predictions = []
    k = 1
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponseMajorityVote(neighbors)
        predictions.append(result)
        print("> predicted=" + repr(result) + ", actual=" + repr(testSet[x][-1]))
    accuracy = getPredictionAccuracy(testSet, predictions)
    print("Accuracy:",accuracy)

In [159]:
main()

Training set: 99
Test set: 50
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-versicolor', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-versicolor', actual='Iris-versicolor'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris-setosa'
> predicted='Iris-virginica', actual='Iris-virginica'
> predicted='Iris-setosa', actual='Iris